In [1]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
import numpy as np
import pandas as pd
from nltk.translate import bleu_score
from nltk.translate.bleu_score import SmoothingFunction

device = 'mps'

processor = TrOCRProcessor.from_pretrained('microsoft/trocr-large-handwritten')
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-large-handwritten').to(device)
model.config.num_beams = 2

/Users/amaljoe/Desktop/Workspace/IITB/NLP/OCR_with_LLMs/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 1024,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 16,
  "num_channels": 3,
  "num_hidden_layers": 24,
  "patch_size": 16,
  "qkv_bias": false,
  "transformers_version": "4.46.2"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRC

In [5]:
def inference(filepath):
    def get_cropped_images(filepath):
        image = Image.open(filepath).convert("RGB")
        
        cropped_images = []
        y = []
        for i in range(700, image.height - 850, 50):
            height  = 150
            cropped_image = image.crop((0, i, image.width, i + height))
            cropped_images.append(cropped_image)
            y.append(i)
            
        return cropped_images, y
            
    def get_model_output(images):
        pixel_values = processor(images=images, return_tensors="pt").pixel_values.to(device)
        output = model.generate(pixel_values, return_dict_in_generate=True, output_scores=True)
        generated_texts = processor.batch_decode(output.sequences, skip_special_tokens=True)
        return generated_texts, output.sequences_scores
    
    def get_qualified_texts(generated_texts, scores, y):
        qualified_texts = []
        for text, score, y_i in zip(generated_texts, scores, y):
            if score > 0.8:
                qualified_texts.append({
                    'text': text,
                    'score': score,
                    'y': y_i
                })
        for i in range(len(qualified_texts)):
            hyp = qualified_texts[i]['text'].split()
            bleu = 0
            if i < len(qualified_texts) - 1:
                ref = qualified_texts[i + 1]['text'].split()
                bleu = get_bleu_score(hyp, [ref])
            qualified_texts[i]['bleu'] = bleu

        final_texts = []
        new = True
        for i in range(len(qualified_texts)):
            if new:
                final_texts.append(qualified_texts[i])
            else:
                if final_texts[-1]['score'] < qualified_texts[i]['score']:
                    final_texts[-1] = qualified_texts[i]
            new = qualified_texts[i]['bleu'] < 0.5
        return final_texts

    def get_bleu_score(hypothesis, references):
        weights = [0.5, 0.5]
        smoothing = SmoothingFunction()
        return bleu_score.sentence_bleu(references, hypothesis, weights=weights, smoothing_function=smoothing.method1)

    cropped_images, y = get_cropped_images(filepath)
    generated_texts, scores = get_model_output(cropped_images)
    normalised_scores = np.exp(scores.to('cpu').numpy())
    qualified_texts = get_qualified_texts(generated_texts, normalised_scores, y)
    qualified_texts_df = pd.DataFrame(qualified_texts, columns=['text', 'score', 'y'])
    return qualified_texts_df

inference('data/iam_form.png')

/Users/amaljoe/Desktop/Workspace/IITB/NLP/OCR_with_LLMs/.venv/lib/python3.9/site-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


,text,score,y
0,"And , since this is election year in West",0.983949,750
1,"Germany , Dr. Adenauer is in a tough",0.991437,900
2,spot . Joyce Egginton cables : President,0.833769,1100
3,Kennedy at his Washington Press con-,0.971444,1250
4,ference admitted he did not know,0.990958,1450
5,whether America was lagging behind,0.954025,1650
6,Russia in missile power . He said he,0.990940,1800
7,was waiting for his senior military,0.974817,2000
8,aides to come up with the answer on,0.943118,2200
9,"February 20 ,",0.860617,2350
